# QaT Frobenius Test

In the paper [provide reference], we introduce the following primality test:

* (1) Test whether $n$ is a square. If it is, declare $n$ to be composite and stop.
* (2) Set $a=1, 3, 5, \ldots$ such that the Jacobi symbol of $a^2-4$ over $n$ is $-1$, and then set $T=1, 2, 3, \ldots$ and $Q=T^2+aT+1$ such that $Q \neq |a^2-4|$ and the Jacobi symbol of $Q$ over $n$ is $-1$. If $\gcd\left((a^2-4)(a+2T)Q,n\right) \neq 1$ throughout the search, declare $n$ to be composite and stop.
* (3) If $Q^{(n-1)/2} \neq -1 \pmod{n}$, declare $n$ to be composite and stop.
* (4) If $s_n\neq -1 \pmod{n}$ or $t_n\neq a+T\pmod{n}$, declare $n$ to be composite and stop.
* (5) If $n$ is not declared composite in steps (1) to (4), declare $n$ to be a probable prime.

The function <code>QaT_Frobenius_test</code> searches for violations of the underlying test conditions.

In [8]:
import sys
from math import isqrt


def gcd_ext(a, b):
    # returns gcd, x, y, such that gcd = a*x + b*y
    if a == 0:
        return b, 0, 1
    d, x, y = gcd_ext(b % a, a)
    return d, y - (b // a) * x, x


def jacobi(a, n):
    # Jacobi symbol (generalized Legendre symbol) for positive, odd n
    a = a % n
    t = 1
    while a != 0:
        while a % 2 == 0:
            a = a // 2
            r = n % 8
            if r == 3 or r == 5:
                t = -t
        a, n = n, a
        if a % 4 == 3 and n % 4 == 3:
            t = -t
        a = a % n
    if n == 1:
        return t
    else:
        return 0


def bpsw_probable_prime(n):
    # Strong Fermat base 2 test + strong Lucas test using method A from https://arxiv.org/pdf/2006.14425, deterministic test up to 2^64 > 10^15
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    is_probable_prime = False
    if pow(2, d, n) == 1:
        is_probable_prime = True
    for r in range(s):
        if pow(2, d * pow(2, r), n) == n - 1:
            is_probable_prime = True
            break
    D = 5
    while jacobi(D, n) != -1:
        D -= 2 * D + 2 * (1 if D > 0 else -1)
    Q = (1 - D) // 4
    out = [1, 1]
    d = n + 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1
    for b in range((n + 1).bit_length() - 2, -1, -1):
        U = out[0]
        V = out[1]
        if b == s:
            if U % n == 0:
                is_probable_prime = False
                break
        if b <= s:
            if V % n == 0:
                is_probable_prime = False
                break
        if (n >> b) & 1 == 1:
            out[0] = (U * V) % n
            out[1] = ((D * U * U + V * V) * ((n + 1) // 2)) % n
            U = out[0]
            V = out[1]
            out[0] = ((U + V) * ((n + 1) // 2)) % n
            out[1] = ((D * U + V) * ((n + 1) // 2)) % n
        else:
            out[0] = (U * V) % n
            out[1] = ((D * U * U + V * V) * ((n + 1) // 2)) % n
    return is_probable_prime


def calc_s_t(a, T, n):
    out = [1, T]
    for b in range(n.bit_length() - 2, -1, -1):
        s = out[0]
        t = out[1]
        if (n >> b) & 1 == 1:
            # doubling
            out[0] = (s * (a * s + 2 * t)) % n
            out[1] = ((t - s) * (t + s)) % n

            # incrementing
            s = out[0]
            t = out[1]
            out[0] = ((a + T) * s + t) % n
            out[1] = (T * t - s) % n
        else:
            # doubling
            out[0] = (s * (a * s + 2 * t)) % n
            out[1] = ((t - s) * (t + s)) % n
    return out

In [9]:
def QaT_Frobenius_test(low=0, high=0):
    low = max(15, low)
    if high == 0:
        high = low
    for n in range(low, high + 1):
        is_composite = False
        if n % 2 == 0:
            continue
        if pow(isqrt(n), 2) == n:
            continue
        if bpsw_probable_prime(n):
            continue
        a = 1
        D = -3
        if abs(gcd_ext(D, n)[0]) != 1:
            continue
        while jacobi(D, n) != -1:
            D += 4 * a + 4
            a += 2
            if abs(gcd_ext(D, n)[0]) != 1:
                is_composite = True
                break
        T = 1
        Q = T * T + a * T + 1
        if abs(gcd_ext((2 * T + a) * Q, n)[0]) != 1:
            continue
        while not is_composite and (Q == abs(D) or jacobi(Q, n) != -1):
            Q += 2 * T + 1 + a
            T += 1
            if abs(gcd_ext((2 * T + a) * Q, n)[0]) != 1:
                is_composite = True
        if not is_composite:
            if pow(Q, (n - 1) // 2, n) == n - 1:
                print("Q " + str(n))
            s_t = calc_s_t(a, T, n)
            if s_t[0] % n == n - 1:
                print("sn " + str(n))
            if (s_t[1] - a - T) % n == 0:
                print("tn " + str(n))

In [10]:
sn = [7827219287, 30371119094359, 63891422400971, 134483747727349]
tn = [
    6500797,
    118204297,
    76292030887,
    132308954471,
    797731655753,
    50360298471893,
    825047294702087,
]

for n in sn:
    QaT_Frobenius_test(n)
for n in tn:
    QaT_Frobenius_test(n)

sn 7827219287
sn 30371119094359
sn 63891422400971
sn 134483747727349
tn 6500797
tn 118204297
tn 76292030887
tn 132308954471
tn 797731655753
tn 50360298471893
tn 825047294702087
